**Для этого исследования банк представил данные - статистику платёжеспособности клиентов. Необходимо выяснить влияют ли количество детей и семейное положение на факт своевременнего погашения кредита. Предпологается, что результаты данного исследования будут включены в модель банковского скоринга.**

** For this study, the bank provided data - statistics on the solvency of customers. It is necessary to find out whether the number of children and marital status affect the fact of timely repayment of the loan. It is expected that the results of this study will be included in the bank scoring model.**

### Шаг 1. Изучение представленных данных.
### Step 1. Examine the presented data.

In [1]:
# Импортируем библиотеку "Пандас" и загружаем исходный файл. 
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

In [2]:
# Ознакомимся с первыми 10-ю строками.
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# Взглянем на общую информацию о содержании таблицы.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Разберемся с пропущенными значениями. Видно, что пропущенные значение есть только в стобцах
'total_income' и 'days_employed'. Причем, количество пропущенных значений одинаково.

Let's deal with the missing values. It can be seen that the missing value is only in the columns
'total_income' and 'days_employed'. Moreover, the number of missing values is the same.

In [4]:
# Сравним индексы строк с пропущенными значениями в вышеуказанных столбцах.
if df[df['days_employed'].isna() == True].index.tolist() == df[df['total_income'].isna() == True].index.tolist():
    print('Строки с пропущенными значениями в столбцах "days_employed" и "total_income" одни и те же.')
else: 
    print('Строки с пропущенными значениями в столбцах "days_employed" и "total_income" разные.')

Строки с пропущенными значениями в столбцах "days_employed" и "total_income" одни и те же.


The rows with missing values in the "days_employed" and "total_income" columns are the same.

In [5]:
df[df['days_employed'].isna() == True].groupby('income_type')['purpose'].count() / df.groupby('income_type')['purpose'].count()

income_type
безработный             NaN
в декрете               NaN
госслужащий        0.100754
компаньон          0.099902
пенсионер          0.107106
предприниматель    0.500000
сотрудник          0.099379
студент                 NaN
Name: purpose, dtype: float64

Какой-то закономерности в пропусах не видно. 
Выше вы можете видеть коэффициент полученный делением количества наблюдений с пропусками на обшее количество наблюдений по группам населения.
В отличии от социаологических опрсов, где респондент может уклониться от ответа о доходе, в случае с заявкой на кредит,
клиент не может не сообщить о своем доходе. Другой вопрос - правидива ли предсталвенная клиетом информация. 

No pattern is seen in the gaps.
Above you can see the ratio obtained by dividing the number of missing observations by the total number of observations per population.
Unlike sociological surveys, where the respondent may evade answering about income, in the case of a loan application,
the client cannot but report his income. Another question is whether the information provided by the client is correct.

In [6]:
# Посмотрим сколько какую часть от данных занимают строки с пропущенными значениями.
print('Объём строк с пропущенными значениями: {:.2%}'.format(len(df[df['days_employed'].isna() == True]) / len(df)))

Объём строк с пропущенными значениями: 10.10%


Volume of rows with missing values: 10.10%

Посмотрим какие уникальные значения содержат столбцы 'education', 'family_status', 'income_type' и
'purpose'.

Let's see what unique values the 'education', 'family_status', 'income_type' and
'purpose'.

In [7]:
# Можно легко заметить, что слова имеют разный регистр, что может вызвать сложности в дальнейшем.
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [8]:
# В этом случе тоже стоит привессти все слова к нижнему регистру
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [9]:
# В этом столбце всё в порядке.
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [10]:
# Видим, что некторые категории, которые по смыслу явлются одинаковыми, описаны по-разному.
# Будем лемматизировать.
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [11]:
# Также  в глаза бросаются отрицательный стаж в днях и, по крайне мере одно, огромное значение.
# Кроме того, данные в этом столбе записаны дробными числами. 
df['days_employed'].head(10)

0     -8437.673028
1     -4024.803754
2     -5623.422610
3     -4124.747207
4    340266.072047
5      -926.185831
6     -2879.202052
7      -152.779569
8     -6929.865299
9     -2188.756445
Name: days_employed, dtype: float64

### Вывод

### Conclusion

Представленные для анлиза данные имеют некторые некритичные изъяны, которые мы легко исправим вдальнейшем.
Удалось установить:
1. Переменные описывающие стаж работы и доход имеют пропуски, причем в одних и тех же строках. Однако их довльно много - 10% от общего объёма наблюдений. Пропуски скорее всего полностью случайны.
2. В столбцах о семейном положении и образовании нет единообразия регистра.
3. В столбце о стаже есть отрицательные и слишком большие значения. 
4. В столбцах о стаже и зарплате представлены дробные значения, которые в этой задачче лучше округлить до целых. 
Отрицательные и дробные значения скорее всего имеют техничекую природу.

The data presented for analysis has some non-critical flaws, which we can easily correct in the future.
Managed to install:
1. Variables describing work experience and income have gaps, and in the same rows. However, there are quite a lot of them - 10% of the total volume of observations. The gaps are most likely completely random.
2. The columns on marital status and education do not have a uniform register.
3. There are negative and too large values ​​in the seniority column.
4. The seniority and salary columns show fractional values, which in this problem should be rounded up to integers.
Negative and fractional values ​​are likely to be of a technical nature.

### Шаг 2. Предобработка данных.

### Step 2. Data preprocessing.

### Обработка пропусков

### Pass handling

Хотя известно, что заполнение пропусков средними искажает характеристики выборки, предпологается,
что этого метода будет достаточно в этой задаче. Так и поступим. Заполним пропущенные значения в 
столбцах стаж и доход средними значениями по этим столбцам.
Можно было удалить строки с пропущенными значениями, но тогда мы бы потряли 10% наблюдений.
Наверно, самым правильным было бы применить регрессию и получить значения снчала для стажа, затем для дохода.

Although it is known that filling in the gaps with means distorts the characteristics of the sample, it is assumed that
that this method will suffice for this task. So let's do it. Fill in the missing values in
the length of service and income columns are the average values for these columns.
It was possible to remove rows with missing values, but then we would lose 10% of the observations.
Probably the most correct would be to apply regression and get the values first for the length of service, then for the income.

Так как стаж не может быть отрицательным, сделаем такие значения положительными.

Since seniority cannot be negative, let's make such values positive.

In [12]:
df['days_employed'] = df['days_employed'].abs()

# Посмотрим результат.
df['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

In [13]:
# Можно заметить сликшом большие значения дней стажа.
# Вряд ли ли кто-то работает дольше чем живет. Найдем такие значения.
df[df['days_employed'] / 365 > df['dob_years']]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


In [14]:
# Найдем медианы стажа для всех групп "income_type".
# Видим что с пенсионерами и безработными что-то не так - слигком большие знаечния стажа.
medians = df.groupby('income_type')['days_employed'].median()
medians

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

Все значения стажа пенсионеров и безработных заполним нулями,
так как на данный момент они не трудоусроенны. Именно это инетресует банк при выдачи кредита.
Видимо большие значения стажа у этих групп населения можно объяснить технической ошибкой.

All values of the seniority of pensioners and the unemployed will be filled with zeros,
as they are currently unemployed. This is what the bank asks when issuing a loan.
Apparently, the large values of experience in these population groups can be explained by a technical error.

Пропуски по остальным группам заполним медианами.

Fill in the gaps for the rest of the groups with medians.

In [15]:
df['days_employed'] = df.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.median()))
df.loc[(df['income_type'] == 'безработный'), 'days_employed'] = 0
df.loc[(df['income_type'] == 'пенсионер'), 'days_employed'] = 0

In [16]:
# Проверим медианы теперь.
df.groupby('income_type')['days_employed'].median()

income_type
безработный           0.000000
в декрете          3296.759962
госслужащий        2689.368353
компаньон          1547.382223
пенсионер             0.000000
предприниматель     520.848083
сотрудник          1574.202821
студент             578.751554
Name: days_employed, dtype: float64

In [17]:
# Посмотрим остались ли нулевые значения в столбце стажа.
df['days_employed'].isna().sum()

0

Теперь посмотрим как обстоят дела с доходами.

Now let's see how things are with income.

In [18]:
# Посмотрим на медианы.
medians_income = df.groupby('income_type')['total_income'].median()
medians_income

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [19]:
# Посмотрим на размах доходов у пенсионеров и безработных, так как до этого мы встрили ошибки в данных стажа по этим группам.
print('Размах в доходах у безработных', df[df['income_type'] == 'безработный']['total_income'].max() 
      - df[df['income_type'] == 'безработный']['total_income'].min())
print('Всего безработных', len(df[df['income_type'] == 'безработный']))
print('Размах в доходах у пенсионеров', df[df['income_type'] == 'пенсионер']['total_income'].max() 
      - df[df['income_type'] == 'пенсионер']['total_income'].min())
print('Всего пенсионеров', len(df[df['income_type'] == 'пенсионер']))

Размах в доходах у безработных 142765.51938439716
Всего безработных 2
Размах в доходах у пенсионеров 714436.0063733842
Всего пенсионеров 3856


The range in income of the unemployed 142765.51938439716

Total unemployed 2

The range in income of pensioners 714436.0063733842

Total pensioners 3856

Обратим внимание, что наблюдений по безработным очень мало - всего два.
Похоже, что проблем с доходами нет: размах отличается от медиан, так что типовыми значениями по пенсионерам стобец с доходами не заполнен. 
Теперь мы можем заполнить заполнить пропуски в доходах соотвествующими медианами по группам.

Note that there are very few observations on the unemployed - only two.
It seems that there are no problems with income: the range differs from the median, so the income column is not filled with typical values for pensioners.
We can now fill in the income gaps with the appropriate group medians.

In [20]:
df['total_income'] = df.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

In [21]:
# Проверим результаты.
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [22]:
# Так же обратим внимание, что в стобце 'children' есть значение "-1". Предположу, что это ошибка и подарузмаевтся "1".
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [23]:
# Заменим такие значения на "1".
df.loc[(df['children'] == -1), 'children'] = 1

### Вывод

### Conclusion

1. Данных по безработным очень мало - всего 2 наблюдения.
2. Значения стажа у безработных и пенсионеров ошибочны. 
На этом шаге я заполнил значения стажа у безработных и пенсионеров нулями, так как статус этих людей подразумевают, что на данный момент они не равботают, а правилных значений у нас нет вообще.
Пропущенные значения доходов, я заполнил медианами по соответсвующим группам.

1. There are very few data on the unemployed - only 2 observations.
2. The values of the length of service for the unemployed and pensioners are erroneous.
At this step, I filled in the seniority values for the unemployed and pensioners with zeros, since the status of these people implies that they are not currently working, and we do not have the correct values \u200b\u200bat all.
I filled in the missing income values with medians for the respective groups.

### Замена типа данных

### Replacing the data type

1. Заменим education_id и family_status_id на тип object, так как эти стобцы несут в себе смысл кодировки других строковых значений. debt приведем к типу bool.
2. Округлим значения в 'days_employed' и 'days_employed' и приведем к формату int, так занкми после запятой в этих стобцах не имеют значения.



1. Let's replace education_id and family_status_id with the object type, since these columns carry the meaning of encoding other string values. debt will be cast to bool.
2. Round the values in 'days_employed' and 'days_employed' and convert them to int format, so the decimal places in these columns don't matter.

In [24]:
# Выполним пункт 1 и помтрим на результат.
df['education_id'] = df['education_id'].astype('object')
df['family_status_id'] = df['family_status_id'].astype('object')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null object
family_status       21525 non-null object
family_status_id    21525 non-null object
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(3), object(7)
memory usage: 2.0+ MB


In [25]:
# Выполним пункт 2 и посмотрим на результат.
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

### Conclusion

Мы исправили почти все проблемы которые были в таблице: заполнили пропуски на прошлом этапе и привели столбцы к нужным типам. Осталось только сделать регистр однародным.

We fixed almost all the problems that were in the table: we filled in the gaps at the last stage and cast the columns to the required types. It remains only to make the register homogeneous.

### Обработка дубликатов

### Handling duplicates

В-первую очередь приведем все данные к нижнему регистру. Затем найдем дубликаты и удалим.

First of all, we will bring all the data to lower case. Then we find duplicates and delete them.

In [26]:
# Приводим к нижнему регистру стобцы со строковм типом данных.
df['education'] = df['education'].astype(str).str.lower()
df['family_status'] = df['family_status'].astype(str).str.lower()
df['gender'] = df['gender'].astype(str).str.lower()
df['income_type'] = df['income_type'].astype(str).str.lower()
df['purpose'] = df['purpose'].astype(str).str.lower()

In [27]:
df[df.duplicated() == True]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,f,сотрудник,0,142594,покупка жилья для семьи
3290,0,0,58,среднее,1,гражданский брак,1,f,пенсионер,0,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,f,сотрудник,0,142594,свадьба
4851,0,0,60,среднее,1,гражданский брак,1,f,пенсионер,0,118514,свадьба
5557,0,0,58,среднее,1,гражданский брак,1,f,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,0,64,среднее,1,женат / замужем,0,f,пенсионер,0,118514,дополнительное образование
21032,0,0,60,среднее,1,женат / замужем,0,f,пенсионер,0,118514,заняться образованием
21132,0,1574,47,среднее,1,женат / замужем,0,f,сотрудник,0,142594,ремонт жилью
21281,1,1574,30,высшее,0,женат / замужем,0,f,сотрудник,0,142594,покупка коммерческой недвижимости


In [28]:
# Удалим дубликаты.
df.drop_duplicates(inplace=True)

In [29]:
# Восстановим индексы.
df.reset_index(drop=True, inplace=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null object
family_status       21454 non-null object
family_status_id    21454 non-null object
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(5), object(7)
memory usage: 2.0+ MB


### Вывод

### Conclusion

Дубликаты могли появится двумя путями: задвоение в учете либо один и тот же человек взял кредит несколько раз. Проверить вторую гипотизу можно имея либо id клиента либо даты сбора наблюдения. На данный момент это проверить не получится, так как клиент мог родить ребенка, сменить работу, жениться и т.д, при этом мы не знаем сколько времени прошло между заявками.

Duplicates could appear in two ways: double accounting or the same person took out a loan several times. You can check the second hypothesis by having either the client id or the date the observation was collected. At the moment, this cannot be verified, since the client could give birth to a child, change jobs, get married, etc., while we do not know how much time has passed between applications.

### Лемматизация

### Lemmatization

In [31]:
# Загрузим библиотеу для лемматизации. Лемматизируем цели кредита.
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(' '.join(list(df['purpose'].unique())))

In [32]:
# Считаем количтсво каждой найденной леммы.
from collections import Counter
print(Counter(lemmas))

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


Можно сделать вывод, что люди чаще всего в этой выборке берут кредит на недвижимость, автомобиль, образование и свадьбы.
Заменим длинные наименования целей, на их более лаконичные обозначения.

It can be concluded that people in this sample most often take loans for real estate, cars, education and weddings.
Let's replace the long names of the goals with their more concise designations.

In [33]:
# Напишем функцию, которая будет возвращать цель кредита одним словом.
def purpose(row):
    purpose = row['purpose']
    if "недвиж" in purpose:
        return "недвижимость"
    if "жил" in purpose:
        return "недвижимость"
    if "строит" in purpose:
        return "недвижимость"
    if "образов" in purpose:
        return "образование"
    if "авто" in purpose:
        return "автомобиль"
    if "свадь" in purpose:
        return "свадьба"

In [34]:
# Применим функцию и посомтрим на результат.
df['purpose'] = df.apply(purpose, axis=1)
df['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

### Вывод

### Conclusion

Зачастую цели кредита описываются целыми фразами, которые на самом деле можно свести к одному слову. Так будет легче работать с данными.
В этот раз я применил лемматизацию, что бы узнать наиболее часто встрачающеся слова и затем почти вручную переименовал цели.
Уверен, что можно было привести к исходным формам цели по каждому наблюдению, присвоить каждому слову в наблюдении число, которое бы равнялось частоте использования по выборке в целом (среди существительных, конечно, остальные части речи в каждом наблюдении можно откинуть). Слово с набольшем значением оставлось бы, а остльные удалялись, и так по каждому наблюдению. Но я так пока не научился. 

Often, the purpose of the loan is described in whole phrases, which can actually be reduced to one word. This will make it easier to work with the data.
This time I applied lemmatization to find out the most frequently occurring words and then almost manually renamed the targets.
I am sure that it was possible to bring to the original forms of the goal for each observation, to assign to each word in the observation a number that would equal the frequency of use in the sample as a whole (among nouns, of course, the remaining parts of speech in each observation can be discarded). The word with the largest value would remain, and the rest would be deleted, and so on for each observation. But I haven't learned that yet.

### Категоризация данных

### Data categorization

Для ответов на нижепоставленные вопросы нам не хватает только разделения заещиков по доходам.
Введем следующие классы:

-'меньше среднего' доход менее 60000;

-'средний класс' доход более или равен 60000, но менее 150000;

-'выше среднего'доход более или равен 150000, но менее 500000;

-'обеспеченные' доход более 500000.

To answer the questions below, we lack only the division of borrowers by income.
Let's introduce the following classes:

-'less than average' income less than 60,000;

- 'middle class' income is greater than or equal to 60,000 but less than 150,000;

- 'higher than average' income greater than or equal to 150,000 but less than 500,000;

- 'secured' income over 500,000.



In [35]:
# Напишем функцию для категоризации по доходам и применим ее.
def type_income(row):
    total_income = row['total_income']
    try:
        if total_income < 60000:
            return 'меньше среднего'
        elif 60000 <= total_income < 150000:
            return 'средний класс'
        elif 150000 <= total_income < 500000:
            return 'выше среднего'
        elif total_income >= 500000:
            return 'обеспеченные'
    except:
        return 'значение не является числом'
df['income_type'] = df.apply(type_income, axis=1)

In [36]:
# Посмотрим на результат.
df['income_type'].unique()

array(['выше среднего', 'средний класс', 'меньше среднего',
       'обеспеченные'], dtype=object)

Разделим заемщиков на квартили.

Dividing borrowers into quartiles.

In [37]:
df['income_quartiles'] = pd.qcut(df['total_income'], 4)

### Вывод

### Conclusion

Чтобы понять как влиют доходы на возвращаемость, было необходимо придумать какую-то классификацию заемщиков по доходам. Классификация по доходам на 4 группы, показалась мне наиболее подходящей.

To understand how income affects repayment, it was necessary to come up with some sort of classification of borrowers by income. Classification by income into 4 groups seemed to me the most appropriate.

### Шаг 3. Ответы на вопросы.
### Step 3. Answering questions.

Для начала выясним какой процент возврата в общем по выборке.

To begin with, let's find out what percentage of the return is in the total for the sample.

In [38]:
print('{:.2%}'.format(len(df[df['debt'] == True]) / len(df['debt'])), 'заемщиков не вернули кредит в срок.')

8.12% заемщиков не вернули кредит в срок.


8.12% of borrowers did not repay the loan on time.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

- Is there a relationship between having children and repaying the loan on time?

In [39]:
pivot_children = df.pivot_table(index='children', values='debt', aggfunc=lambda x: str(round(x.sum() / x.count() * 100, 2)) 
                                + '%')
print('Процент невозврата')
print(pivot_children)

Процент невозврата
            debt
children        
0          7.54%
1          9.17%
2          9.45%
3          8.18%
4          9.76%
5           0.0%
20        10.53%


In [40]:
print('Заемщиков с количеством детей больше 4-ех:', df[df['children'] > 4 ]['children'].count())

Заемщиков с количеством детей больше 4-ех: 85


Borrowers with more than 4 children: 85

### Вывод

### Conclusion

Заемщики без детей чаще возврашают кредиты в срок. Заемщики с 5 детьми и больше - довольно редкое являние, поэтому на эти значения в этой задаче можно не обращать внимание.

Borrowers without children are more likely to repay loans on time. Borrowers with 5 or more children are quite rare, so these values can be ignored in this problem.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

- Is there a relationship between marital status and loan repayment on time?

In [41]:
pivot_family_status = df.pivot_table(index='family_status', values='debt', aggfunc=lambda x: str(round(x.sum() / x.count() 
                                                                                                       * 100, 2)) + '%')
print('Процент невозврата')
print(pivot_family_status)

Процент невозврата
                        debt
family_status               
в разводе              7.11%
вдовец / вдова         6.57%
гражданский брак       9.35%
женат / замужем        7.55%
не женат / не замужем  9.75%


### Вывод

### Conclusion

Заемщики никогда не состоявшие в официальном браке реже возвращают кредиты в срок.

Borrowers who have never been officially married are less likely to repay loans on time.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

- Is there a relationship between the level of income and repayment of the loan on time?

In [42]:
pivot_income = df.pivot_table(index='income_type', values='debt', aggfunc=lambda x: str(round(x.sum() / x.count() 
                                                                                              * 100, 2)) + '%')
print('Процент невозврата по интутивно выбранным категориям дохода заемщиков')
print(pivot_income)

Процент невозврата по интутивно выбранным категориям дохода заемщиков
                  debt
income_type           
выше среднего    7.79%
меньше среднего  6.08%
обеспеченные     6.31%
средний класс    8.59%


In [43]:
pivot_income_quartiles = df.pivot_table(index='income_quartiles', values='debt', aggfunc=lambda x: str(round(x.sum() / x.count() 
                                                                                                             * 100, 2)) + '%')                                                                                                       
print('Процент невозврата по квартилям доходов заемщиков')                                                                                                             
print(pivot_income_quartiles)

Процент невозврата по квартилям доходов заемщиков
                         debt
income_quartiles             
(20666.999, 107623.0]   7.96%
(107623.0, 142594.0]    8.82%
(142594.0, 195820.25]   8.54%
(195820.25, 2265604.0]  7.14%


### Вывод

### Conclusion

Наиболее обеспеченные и наименние обеспеченные заемщики возвращают кредиты в срок чаще, чам представители средних классов. Это подверждается двумя типами категоризации: интуитвным и квартильным.

The wealthiest and least wealthy borrowers repay loans on time more often than the middle classes. This is supported by two types of categorization: intuitive and quartile.

- Как разные цели кредита влияют на его возврат в срок?

- How do different purposes of the loan affect its repayment on time?

In [44]:
pivot_purpose = df.pivot_table(index='purpose', values='debt', aggfunc=lambda x: str(round(x.sum() / x.count() 
                                                                                           * 100, 2)) + '%')
print('Процент невозврата')
print(pivot_purpose)

Процент невозврата
               debt
purpose            
автомобиль    9.36%
недвижимость  7.23%
образование   9.22%
свадьба        8.0%


### Вывод

### Conclusion

Возврат кредита вероятнее, если он взят на покупку недвижимости или организации свадьбы.

Repayment of a loan is more likely if it is taken to purchase real estate or organize a wedding.

### Шаг 4. Вывод.

### Step 4. Conclusion.

Заемщиков можно описать переменными, а переменные могут подсказать будет ли долг возвращен. В этом документе мы показали как некоторые параметры влияют на возврат кредита. Лучшего результата и больших отклонений от среднего процента возвращаймости кредита, возможно, можно добиться комбинированием перменных. 
А пока нам ивзестно, что крдиты лучше возвращают люди, которые:
- состоят или состояли в браке;

- не имеют детей;

- зарабатывают меньше 60000 либо больше 500000;

- берут кредит на недвидимость или свадбу.

Однако критичных отколнений по группам не выявленно.

Borrowers can be described by variables, and variables can tell whether the debt will be repaid. In this document, we have shown how some parameters affect the return of a loan. A better result and larger deviations from the average loan recovery rate can probably be achieved by combining variables.
In the meantime, we know that credits are better returned by people who:
- are or have been married;

- do not have children;

- earn less than 60,000 or more than 500,000;

- take a loan for real estate or a wedding.

However, there were no critical deviations by groups.

На предыдущих шагах мы подготовили данные:

- привели столбы к подоходящим форматам;

- заполнили пропуски в доходах и стаже медианами;

- исправили невозможные значения стажа у пенсионеров и безработных на нули;

- исправили отрицательные значения количества детей на положительные;

- привели все столбцы к нижнему регистру;

- разделили заемщиков на 4 категории по доходам и добавили альтернативное разделение заемщиков на квартили доходов;

- провели лемматизацию целей кредита; уницифировали цели при помощи выявленных лемм.

In the previous steps, we prepared the data:

- brought the pillars to suitable formats;

- filled gaps in income and length of service with medians;

- fixed the impossible values ​​of seniority for pensioners and the unemployed to zero;

- corrected the negative values of the number of children to positive;

- brought all columns to lower case;

- divided borrowers into 4 income categories and added an alternative division of borrowers into income quartiles;

- carried out lemmatization of the purposes of the loan; unified the goals with the help of the identified lemmas.